In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.1.1


In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

/home/mbui/miniconda3/envs/data_eng/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/mbui/miniconda3/envs/data_eng/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [12]:
year = 2021
month = 2
df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet')

In [13]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [14]:
## Q1. Notebook
#Run this notebook for the February 2021 FVH data.
#What's the mean predicted duration for this dataset?

print(y_pred.mean())

16.191691679979066


In [15]:
## Q2. Preparing the output
#Like in the course videos, we want to prepare the dataframe with the output. 
#First, let's create an artificial `ride_id` column:

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')


In [16]:
y_pred

array([14.53986486, 13.74042222, 15.59333908, ..., 15.83492293,
       16.78317605, 19.65462607])

In [36]:
frame = {'ride_id': df['ride_id'], 'y_pred': y_pred}
df_result = pd.DataFrame(frame)
#df_result.set_index('ride_id', inplace=True)
print(df_result)

                 ride_id     y_pred
1              2021/02_1  14.539865
2              2021/02_2  13.740422
3              2021/02_3  15.593339
4              2021/02_4  15.188118
5              2021/02_5  13.817206
...                  ...        ...
1037687  2021/02_1037687  12.433246
1037688  2021/02_1037688  12.910885
1037689  2021/02_1037689  15.834923
1037690  2021/02_1037690  16.783176
1037691  2021/02_1037691  19.654626

[990113 rows x 2 columns]


In [37]:
#!pip install pyarrow

In [38]:
output_file = './pred.parquet'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [35]:
df_result.size

990113